# CS161 HW5 (Graph Search)

*Note: these are my own solutions to the problem sets for the the Fall 2017 offering of Stanford's CS161 Design and Analysis of Algorithms. I am not a Stanford student and I do not have access to the solutions. The solutions are not guaranteed to be error-free (but are highly likely to have errors) so use with caution.*

### Links
[Homework 5](http://web.stanford.edu/class/archive/cs/cs161/cs161.1182/Homework/Homework5/HW5.pdf)

[Lecture Notes 9](http://web.stanford.edu/class/archive/cs/cs161/cs161.1182/Lectures/Lecture9/CS161Lecture09.pdf)
(has details of algorithms to which solutions refer)

### Problem 2
#### (a) 

Since all sheep in the same strongly connected component $G$ must have directed paths to each other, essentially need to prove the following to show that all source sheep and only source sheep are part of $G$.

*Proposition*

$$\text{sheep $s$ is source sheep} \Leftrightarrow \text{sheep has directed paths to all } s \in \mathcal{S} = \{s: s \text{ is a source sheep}\}$$

*Proof*

(1) $\implies$
This follows from the definition of a source sheep. In order for sheep $b$ to repost sheep $a$ there must be a a directed path from sheep $a$ to $b$. Since a source sheep must be reposted by all other sheep it must have directed paths to all other sheep, including all other source sheep. (Also every sheep has a directed path to itself of length zero i.e. by posting it 'reposts' itself). 

(2) $\impliedby$
By contradiction. It will suffice to prove that a non-source sheep cannot have a directed path to any source sheep. Suppose a non-source sheep $a$ has a directed path to a souce sheep $s$. Since $s$ by definition must have a directed path to all other sheep and sheep $a$ has a directed path to $s$, sheep $a$ must in turn have a directed path to all other sheep via $s$. But that would make it a source sheep which is a contradiction proving that a non-source sheep cannot have a directed path to any source sheep.

Since we have shown that all source sheep must have directed paths to each other and no non-source sheep can have a directed path to any source sheep, we have shown that all source sheep are in a strongly connected component which contains no non-source sheep. 

#### (b) 
Pseudo-code. Assume that `f` is global dict of finish times.
        
    def find_source(G):
        DFS(G) # O(m + n)
        t_max = 0
        last_finish_node = None
        items = f.items()
        for v, t in items: # O(n)
            if t > t_max:
                t_max = t
                last_finish_node = v
        return last_finish_node
    
Running time is O(n + m) is DFS(G) has running time of O(n + m) and find_last_finish_node has running time of O(n) as it does O(1) work for each of the n nodes. 
    
If a source exists then the node with the largest finish time will be a source. Since a source sheep has directed paths to all other sheep if we run DFS(s) for any source sheep s at any point within the loop in DFS(G) we will visit all remaining unvisited nodes. So at the end of DFS(s), all nodes will have been marked black and there will be no further calls to DFS for any node. Any node u will have a larger finish time than all other nodes marked complete before u in the present call and earlier calls. DFS(v) will mark v complete last (after all other nodes visited during the call). Thus s will have a larger finish time than all other nodes visited during DFS(s) as well as those visited in earlier calls to DFS. As there will be no further calls to DFS, s will have the largest finish time out of all the nodes.

An alternative solution would be to have a variable `source` within the if statement in DFS(G), returning this at the end of DFS(G):

    if colour[v] == 'white':
        t = DFS(v, t)
        t += 1
        node = v
        
Since if v is a source node no other nodes will remain white after calling DFS(v, t), the value of node will not change it will be a source node. 

#### (c) 

Pseudo-code. Assume that nodes do not get modified and that instead attribues like colour, start and finish times are recorded in separate hash tables. This is so that object equality can be used to match nodes across separate runs of DFS.  

    def find_source_if_exists(G):
        maybe_source = find_source(G) # O(m + n)
        t_max = f[maybe_source]
        init(G) # f will be reinitialised now
        DFS(maybe_source, 1) # O(m + n)
        if f[maybe_source] == t_max:  
            return maybe_source
        else:
            return None 
            
Running time is O(m + n) because this is the running time of `find_source` and `DFS(maybe_source, 1)` will take at most the time of `DFS(G)` which is O(m + n). The remaining work done takes time O(1). 
    
If a source node exists then everything in (b) will apply so it will be the node returned by `find_source`. If it is the source node then running DFS starting from it i.e. `DFS(maybe_source, 1)` should lead to it having the largest finish time again since all nodes can be reached from it so the entire graph will have been completed by this one call meaning that `maybe_source` is in fact a source. 

On the other hand if there is no source node in the graph, `maybe_source` will merely be the last node v for which DFS(v) is called in DFS(G) when this is called in `find_source`. So after reinitialising the graph and running DFS starting from `maybe_source`, the finish time of `maybe_source` will not equal `t_max` since all nodes will not be completed at this point. This allows us to deduce that `maybe_source` is not a source. Since we have shown that if a source exists then the node with the highest must a source, this means that no source exists.  

For example in the graph shown below there is no source node. Supposing nodes are prioritised in alphabetical order, the first DFS(P) will be called, which will find {Q, R}, then DFS(S) will be called which will find T.  So S will be returned by `find_source(G)` and we will have that `t_max = f[S] = 10`. However after calling DFS(S, 1), `f[S] = 4 != t_max`. 

<img src ='CS161_HW5_2.png'/>